In [1]:
import sys
sys.path.append('./python')
import TransFTrain
import importlib
importlib.reload(TransFTrain)
import TransFTrain as train

In [2]:
x = train.Tensor([1,2,3], dtype="int8")
y = train.Tensor(x)


In [4]:
x/1

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/root/anaconda3/envs/transftrain/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_874/136630806.py", line 1, in <module>
    x/1
  File "/workspace/workspace/github/TransFTrain/./python/TransFTrain/autograd.py", line 353, in __truediv__
    return TransFTrain.ops.DivScalar(other)(self)
  File "/workspace/workspace/github/TransFTrain/./python/TransFTrain/autograd.py", line 106, in __call__
    return Tensor.make_from_op(self, args)
  File "/workspace/workspace/github/TransFTrain/./python/TransFTrain/autograd.py", line 260, in make_from_op
    tensor.realize_cached_data()
  File "/workspace/workspace/github/TransFTrain/./python/TransFTrain/autograd.py", line 126, in realize_cached_data
    self.cached_data = self.op.compute(
  File "/workspace/workspace/github/TransFTrain/./python/TransFTrain/ops.py", line 110, in compute
NotImplemen

In [15]:
v1 = train.Tensor([0], dtype="float32")
v2 = train.exp(v1)
v3 = v2 + 1
v4 = v2 * v3

In [6]:
y = train.Tensor(x)
print(str(y))

[1 2 3]


In [7]:
print(repr(y))

TransFTrain.Tensor([1. 2. 3.])


In [8]:
import numpy as np

In [10]:
np.array([1,2,3],dtype="float32")

array([1., 2., 3.], dtype=float32)

In [11]:
import numpy as np

In [12]:
np.power(4, 2)

16

In [13]:
a = np.array([2, 3])
a.__pow__(2,3)


NotImplemented

In [17]:
import numpy as np
a = np.array([2, 3])
b = np.array([2, 1])

In [18]:
a/ b

array([1., 3.])

In [19]:
np.divide(a,b)

array([1., 3.])